In [15]:
import pandas as pd
import numpy as np
from geopy.distance import great_circle
from sklearn.cluster import DBSCAN
import re
pd.set_option('display.max_columns', None)

# Primera parte del código
Se genera un Excel con la deuda actualizada y con este código se aplican los filtros necesarios y se limpia el archivo.

In [ ]:
clustering = pd.read_csv("Prueba 2.csv")
clustering['CLIENTE'] = clustering['CLIENTE'].astype(str)
#Se eliminan los números y las K después del guión de la columna CLIENTE
#Se utiliza lambda para que la eliminación se de solo en la columna CLIENTE, de lo contrario también afectaba a las fechas
pattern = r"-?\d+$"
replace_digits = lambda x: re.sub(pattern, "", x)
clustering['CLIENTE'] = clustering['CLIENTE'].apply(replace_digits)
clustering['CLIENTE'] = clustering['CLIENTE'].apply(lambda x: re.sub(r"-[A-Za-z]$", "", x))
#Se agrega una nueva columna SECTOR que toma los 2 primeros de la columna RUTA
clustering.insert(1+1, 'SECTOR', clustering['RUTA'].str[:2])
#Se eliminan todas las filas que contengan la "S" en las columnas solicitadas
quitar_S = ["TIENE CONVENIO", "TIENE BLOQUEO CORTE", "CLIENTE VIP", "TIENE RECLAMO COMERCIAL"]
clustering = clustering[~clustering[quitar_S].apply(lambda row: row.str.contains('S').any(), axis=1)]
#Se generan las condiciones de filtrado que normalmente se requieren para cualquier "todas"
condiciones = [
    ~clustering['F. CORTE CLIENTE'].str.contains('2500', na=False),
    ~clustering['DIRECCION'].str.contains('COSME|PINO|MORRO SOLAR', case=False, na=False),
    ~clustering['NOMBRE'].str.contains('MUNICIPALIDAD', case=False, na=False),
    clustering['DESC. SUCURSAL'] != 'Cañete',
    ~clustering['ANTIGUEDAD SIMULADA'].isin([0, 1, 2]),
    clustering['DISTRITO'] != 'EL AGUSTINO',
    ~clustering['TARIFA'].isin(['BT5E', 'BT6'])
]
#Se aplican las condiciones al dataframe
clustering = clustering
for condition in condiciones:
    clustering = clustering[condition]

# Segunda parte del código
Se generan los clusters

In [7]:
#Se convierten las coordenadas a numpy de forma que podamos realizar operaciones algebraicas
coordenadas = clustering[['COORDENADA DE LATITUD', 'COORDENADA DE LONGITUD']].to_numpy()
#Se define la función de distancia geográfica
def haversine_distance(coord1, coord2):
    return great_circle(coord1, coord2).kilometers
#Se establece el parámetro de distancia, en este caso es 0.8km
eps = 0.8
#Se establece el algoritmo para generar los clusters
db = DBSCAN(eps=eps, min_samples=20, metric=haversine_distance).fit(coordenadas)
#Se le asignan etiquetas a cada cluster
clustering['Cluster'] = db.labels_
#Para visualizar la cantidad de clusters generados
clustering['Cluster'].unique()
#Se genera un nuevo dataframe solo con las columnas que necesitamos
clustering_filtrado = clustering[['CLIENTE', 'RUTA', 'DISTRITO', 'DESC. SUCURSAL', 'DEUDA TOTAL', 'ANTIGUEDAD', 'COORDENADA DE LATITUD', 'COORDENADA DE LONGITUD', 'Cluster']]
#Se genera un archivo csv de este nuevo dataframe
clustering_filtrado.to_csv('Resultado prueba.csv', index=False)
